In [1]:
import torch
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import time

import dataset.dataset as dataset
import datasplit.datasplit as datasplit
import model.models as models
import trainer.trainer as trainer
import utils.utils as utils

torch.cuda.device_count()
cuda0 = torch.device('cuda:0')
cuda1 = torch.device('cuda:1')
cuda2 = torch.device('cuda:2')
cuda3 = torch.device('cuda:3')
device = torch.device(cuda0 if torch.cuda.is_available() else "cpu")

In [2]:
# transforms
transform = transforms.Compose([
        transforms.ToTensor(),
    ])

# dataset
root = 'G:/PracaMagisterska/Dane/Mono_Discret_Audio'
excl_folders = ['MT2']
spectra_folder= 'mel_22050_1024_512'
proc_settings_csv = 'proc_settings.csv'
max_num_settings=3

dataset = dataset.FxDataset(root=root,
                            excl_folders=excl_folders, 
                            spectra_folder=spectra_folder, 
                            processed_settings_csv=proc_settings_csv,
                            max_num_settings=max_num_settings,
                            transform=transform)
dataset.init_dataset()

# split
split = datasplit.DataSplit(dataset, test_train_split=0.8, val_train_split=0.1, shuffle=True)
train_loader, val_loader, test_loader = split.get_split(batch_size=100)

print('dataset size: ', len(dataset))
print('train set size: ', len(split.train_sampler))
print('val set size: ', len(split.val_sampler))
print('test set size: ', len(split.test_sampler))
dataset.fx_to_label

dataset size:  123552
train set size:  88956
val set size:  9885
test set size:  24711


{'808': 0,
 'BD2': 1,
 'BMF': 2,
 'DPL': 3,
 'DS1': 4,
 'FFC': 5,
 'MGS': 6,
 'OD1': 7,
 'RAT': 8,
 'RBM': 9,
 'SD1': 10,
 'TS9': 11,
 'VTB': 12}

In [3]:
# model
input_size = 87  # Update this to match the feature size of your input
hidden_size = 128  # You can adjust this as needed
num_layers = 2  # You can adjust this as needed

setnet = models.SettingsRNN(n_settings=dataset.max_num_settings, input_size=input_size, hidden_size=hidden_size, num_layers=num_layers).to(device)
# optimizer
optimizer_setnet = optim.Adam(setnet.parameters(), lr=0.001)
# loss function
loss_func_setnet = nn.MSELoss(reduction='mean')

print(setnet)
print('Trainable Params: ', sum(p.numel() for p in setnet.parameters() if p.requires_grad))

SettingsRNN(
  (lstm): LSTM(87, 128, num_layers=2, batch_first=True)
  (fc1): Linear(in_features=128, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=3, bias=True)
)
Trainable Params:  266123


In [4]:
# SAVE
models_folder = 'G:/PracaMagisterska/Kod_na_magisterke/gfx-classifier_RNN_version-models_and_results/models'
model_name = '20201111_setnet_mono_disc_best'
results_folder = 'G:/PracaMagisterska/Kod_na_magisterke/gfx-classifier_RNN_version-models_and_results/result'
results_subfolder = '20201111_setnet_mono_disc'

In [5]:
# TRAIN and TEST SetNet OVER MULTIPLE EPOCHS
train_set_size = len(split.train_sampler)
val_set_size = len(split.val_sampler)
test_set_size = len(split.test_sampler)

all_train_losses, all_val_losses, all_test_losses = [],[],[]
all_train_correct, all_val_correct, all_test_correct = [],[],[]
all_train_results, all_val_results, all_test_results = [],[],[]

best_val_correct = 0

start = time.time()

for epoch in range(50):
    train_loss, train_correct, train_results = trainer.train_settings_net(
        model=setnet,
        optimizer=optimizer_setnet, 
        train_loader=train_loader, 
        train_sampler=split.train_sampler, 
        epoch=epoch,
        loss_function=loss_func_setnet, 
        device=device
    )
    
    val_loss, val_correct, val_results = trainer.val_settings_net(
        model=setnet, 
        val_loader=val_loader, 
        val_sampler=split.val_sampler,
        loss_function=loss_func_setnet,
        device=device
    )
    
    test_loss, test_correct, test_results = trainer.test_settings_net(
        model=setnet, 
        test_loader=test_loader, 
        test_sampler=split.test_sampler,
        loss_function=loss_func_setnet,
        device=device
    )
    # save model
    if val_correct > best_val_correct:
        best_val_correct = val_correct
        torch.save(setnet, '%s/%s' % (models_folder, model_name))
        print('\n=== saved best model ===\n')
        
    # append results    
    all_train_losses.append(train_loss)
    all_val_losses.append(val_loss)
    all_test_losses.append(test_loss)
    
    all_train_correct.append(train_correct)
    all_val_correct.append(val_correct)
    all_test_correct.append(test_correct)
    
    all_train_results.append(train_results)
    all_val_results.append(val_results)
    all_test_results.append(test_results)

stop = time.time()
print(f"Training time: {stop - start}s")

Train Epoch: 0	[5000/88956 (6%)]	Total Loss: 13.7287	Avg Loss: 0.0027
Train Epoch: 0	[10000/88956 (11%)]	Total Loss: 20.7154	Avg Loss: 0.0021
Train Epoch: 0	[15000/88956 (17%)]	Total Loss: 27.3282	Avg Loss: 0.0018
Train Epoch: 0	[20000/88956 (22%)]	Total Loss: 34.1931	Avg Loss: 0.0017
Train Epoch: 0	[25000/88956 (28%)]	Total Loss: 40.8138	Avg Loss: 0.0016
Train Epoch: 0	[30000/88956 (34%)]	Total Loss: 47.5428	Avg Loss: 0.0016
Train Epoch: 0	[35000/88956 (39%)]	Total Loss: 54.2807	Avg Loss: 0.0016
Train Epoch: 0	[40000/88956 (45%)]	Total Loss: 61.1579	Avg Loss: 0.0015
Train Epoch: 0	[45000/88956 (51%)]	Total Loss: 67.6693	Avg Loss: 0.0015
Train Epoch: 0	[50000/88956 (56%)]	Total Loss: 74.3636	Avg Loss: 0.0015
Train Epoch: 0	[55000/88956 (62%)]	Total Loss: 80.9484	Avg Loss: 0.0015
Train Epoch: 0	[60000/88956 (67%)]	Total Loss: 87.8459	Avg Loss: 0.0015
Train Epoch: 0	[65000/88956 (73%)]	Total Loss: 94.5395	Avg Loss: 0.0015
Train Epoch: 0	[70000/88956 (79%)]	Total Loss: 101.4716	Avg Loss: 

In [19]:
# BEST RESULTS
print('Accuracy: ', 100 * max(all_train_correct) / train_set_size)
print('Epoch: ', np.argmax(all_train_correct))
print()
print('Accuracy: ', 100 * max(all_val_correct) / val_set_size)
print('Epoch: ', np.argmax(all_val_correct))
print()
print('Accuracy: ', 100 * max(all_test_correct) / test_set_size)
print('Epoch: ', np.argmax(all_test_correct))
print()

Accuracy:  59.093259589010295
Epoch:  49

Accuracy:  57.51138088012139
Epoch:  49

Accuracy:  58.613572902755855
Epoch:  49



In [27]:
# SAVE RESULTS - all losses, all correct, best results
import pickle 

all_train_losses_npy = np.array(all_train_losses)
all_train_correct_npy = np.array(all_train_correct)

all_val_losses_npy = np.array(all_val_losses)
all_val_correct_npy = np.array(all_val_correct)

all_test_losses_npy = np.array(all_test_losses)
all_test_correct_npy = np.array(all_test_correct)

# Define the paths for saving
train_losses_path = '%s/%s/%s.pkl' % (results_folder, results_subfolder, 'all_train_losses')
train_correct_path = '%s/%s/%s.pkl' % (results_folder, results_subfolder, 'all_train_correct')
best_train_results_path = '%s/%s/%s.pkl' % (results_folder, results_subfolder, 'best_train_results')

val_losses_path = '%s/%s/%s.pkl' % (results_folder, results_subfolder, 'all_val_losses')
val_correct_path = '%s/%s/%s.pkl' % (results_folder, results_subfolder, 'all_val_correct')
best_val_results_path = '%s/%s/%s.pkl' % (results_folder, results_subfolder, 'best_val_results')

test_losses_path = '%s/%s/%s.pkl' % (results_folder, results_subfolder, 'all_test_losses')
test_correct_path = '%s/%s/%s.pkl' % (results_folder, results_subfolder, 'all_test_correct')
best_test_results_path = '%s/%s/%s.pkl' % (results_folder, results_subfolder, 'best_test_results')

fx_labels_path = '%s/%s/%s.pkl' % (results_folder, results_subfolder, 'fx_labels')

# Save the training data using pickle
with open(train_losses_path, 'wb') as f:
    pickle.dump(all_train_losses, f)

with open(train_correct_path, 'wb') as f:
    pickle.dump(all_train_correct, f)

with open(best_train_results_path, 'wb') as f:
    pickle.dump(all_train_results[28], f)

# Save the validation data using pickle
with open(val_losses_path, 'wb') as f:
    pickle.dump(all_val_losses, f)

with open(val_correct_path, 'wb') as f:
    pickle.dump(all_val_correct, f)

with open(best_val_results_path, 'wb') as f:
    pickle.dump(all_val_results[28], f)

# Save the test data using pickle
with open(test_losses_path, 'wb') as f:
    pickle.dump(all_test_losses, f)

with open(test_correct_path, 'wb') as f:
    pickle.dump(all_test_correct, f)

with open(best_test_results_path, 'wb') as f:
    pickle.dump(all_test_results[28], f)

# Save the fx_labels using pickle
with open(fx_labels_path, 'wb') as f:
    pickle.dump(list(dataset.fx_to_label.keys()), f)